## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd
import pickle as pkl

In [2]:
df=pd.read_csv('dataset/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


<ul>
 <li>1: unreliable</li>
 <li>0: reliable</li>
</ul>

In [4]:
###Drop Nan Values
df=df.dropna()

In [5]:
## Get the Independent Features
X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
X.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.1.0'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [13]:
messages=X.copy()

In [14]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [16]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\S
[nltk_data]     Ponraj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Apply Stemming on text data
<b> Stemming takes long time on large data</b>

In [19]:
try:
    ## Try to load saved corpus
    with open('corpus.pkl', 'rb') as f:
        corpus = pkl.load(f)
except:
    ## If not saved before create a corpus
    
    ### Dataset Preprocessing
    from nltk.stem.porter import PorterStemmer
    ps = PorterStemmer()
    corpus = []
    for i in range(0, len(messages)):
        review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
        review = review.lower()
        review = review.split()

        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)
        corpus.append(review)

    with open('corpus.pkl', 'wb') as f:
        pkl.dump(corpus, f)

In [38]:
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4524, 1665, 712, 1817, 2906, 2268, 3683, 3993, 2912, 4920],
 [988, 1821, 1904, 4306, 3178, 1685, 111],
 [3269, 3526, 782, 2987],
 [934, 4432, 3880, 4809, 1443, 3118],
 [2127, 3178, 2133, 250, 1711, 1291, 3178, 3202, 3080, 2492],
 [2607,
  998,
  4192,
  4988,
  3430,
  4028,
  3623,
  1898,
  1681,
  547,
  4777,
  2770,
  3274,
  1292,
  111],
 [4710, 4726, 4063, 1365, 1551, 55, 2874, 1430, 2991, 4175, 4862],
 [968, 872, 4861, 3222, 1279, 3705, 4028, 4423, 2991, 4175, 4862],
 [4709, 3994, 454, 955, 2607, 1480, 3138, 1048, 4028, 4898],
 [2887, 2316, 4622, 1966, 4886, 2598, 734, 2077],
 [4295, 1754, 1132, 932, 4655, 1100, 3521, 3569, 3937, 4962, 3086],
 [4809, 4107, 2906, 1480, 4028, 1279],
 [2528, 3407, 4819, 3682, 1102, 2182, 1558, 4807, 2957],
 [4291, 1304, 2620, 397, 3368, 1137, 996, 2991, 4175, 4862],
 [241, 239, 4841, 2812, 380, 2991, 4175, 4862],
 [1258, 2653, 2229, 4104, 1493, 2763, 2866, 2866, 4283, 3416],
 [4486, 4483, 1821],
 [4481, 4896, 338, 231, 4028, 351, 531, 111],
 [4

### Embedding Representation

In [22]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3993 2912 4920]
 [   0    0    0 ... 3178 1685  111]
 [   0    0    0 ... 3526  782 2987]
 ...
 [   0    0    0 ... 2991 4175 4862]
 [   0    0    0 ... 1818 3335 1496]
 [   0    0    0 ... 2193  539 4157]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4524,
       1665,  712, 1817, 2906, 2268, 3683, 3993, 2912, 4920])

## Model Building

In [24]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
len(embedded_docs),y.shape

(18285, (18285,))

In [26]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [27]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [29]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 12s 1ms/sample - loss: 0.3424 - accuracy: 0.8424 - val_loss: 0.1987 - val_accuracy: 0.9148
Epoch 2/10
12250/12250 [==============================] - 4s 340us/sample - loss: 0.1430 - accuracy: 0.9432 - val_loss: 0.2016 - val_accuracy: 0.9183
Epoch 3/10
12250/12250 [==============================] - 5s 413us/sample - loss: 0.0975 - accuracy: 0.9624 - val_loss: 0.2240 - val_accuracy: 0.9170
Epoch 4/10
12250/12250 [==============================] - 5s 390us/sample - loss: 0.0700 - accuracy: 0.9757 - val_loss: 0.2536 - val_accuracy: 0.9176
Epoch 5/10
12250/12250 [==============================] - 5s 441us/sample - loss: 0.0472 - accuracy: 0.9851 - val_loss: 0.2723 - val_accuracy: 0.9162
Epoch 6/10
12250/12250 [==============================] - 5s 447us/sample - loss: 0.0329 - accuracy: 0.9888 - val_loss: 0.3403 - val_accuracy: 0.9153
Epoch 7/10
12250/12250 [============================

### Adding Dropout 

In [30]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model_new=Sequential()
model_new.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model_new.add(Dropout(0.3))
model_new.add(LSTM(100))
model_new.add(Dropout(0.3))
model_new.add(Dense(1,activation='sigmoid'))
model_new.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_new.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 9s 747us/sample - loss: 0.3583 - accuracy: 0.8262 - val_loss: 0.2021 - val_accuracy: 0.9170
Epoch 2/10
12250/12250 [==============================] - 5s 384us/sample - loss: 0.1499 - accuracy: 0.9387 - val_loss: 0.1950 - val_accuracy: 0.9225
Epoch 3/10
12250/12250 [==============================] - 6s 450us/sample - loss: 0.1115 - accuracy: 0.9571 - val_loss: 0.2090 - val_accuracy: 0.9167
Epoch 4/10
12250/12250 [==============================] - 5s 428us/sample - loss: 0.0864 - accuracy: 0.9680 - val_loss: 0.2299 - val_accuracy: 0.9203
Epoch 5/10
12250/12250 [==============================] - 5s 398us/sample - loss: 0.0700 - accuracy: 0.9764 - val_loss: 0.2460 - val_accuracy: 0.9181
Epoch 6/10
12250/12250 [==============================] - 5s 443us/sample - loss: 0.0531 - accuracy: 0.9825 - val_loss: 0.2997 - val_accuracy: 0.9170
Epoch 7/10
12250/12250 [===========================

### Performance Metrics And Accuracy

In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [35]:
def model_Validate(local_model):
    y_pred=local_model.predict_classes(X_test)
    print('Confusion matrix: \n', confusion_matrix(y_test,y_pred))
    print('Accuracy: ',accuracy_score(y_test,y_pred))

In [36]:
model_Validate(model)

Confusion matrix: 
 [[3073  346]
 [ 164 2452]]
Accuracy:  0.9154929577464789


In [37]:
model_Validate(model_new)

Confusion matrix: 
 [[3122  297]
 [ 213 2403]]
Accuracy:  0.9154929577464789
